In [1]:
#----------------------------------------------------------------------------
# Created By  : Leonardo Citraro leonardo.citraro@epfl.ch
# Date: 2021
# ---------------------------------------------------------------------------
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import sys
import mot3d
import mot3d.weight_functions as wf
from mot3d.utils import utils


In [8]:

print("Loading detections..")

ps1 = np.array(utils.json_read("/home/jash/Desktop/mot3d/examples/simple_2d/track_line1.json"))
ps2 = np.array(utils.json_read("/home/jash/Desktop/mot3d/examples/simple_2d/track_line2.json"))

# Plot ps1 and ps2
plt.figure()
plt.plot(ps1[:,0], ps1[:,1], 'r.')
plt.plot(ps2[:,0], ps2[:,1], 'b.')
plt.xlabel('x-axis')
plt.ylabel('y-axis')
plt.title("orignal detections")
plt.savefig("results/orignal.jpg")

ps2 = ps2-np.array([[4,0]])

ps1[4] = ps1[4]+np.random.randn(2)
ps2[4] = ps2[4]+np.random.randn(2) 

plt.figure()
plt.plot(ps1[:,0], ps1[:,1], 'r.')
plt.plot(ps2[:,0], ps2[:,1], 'b.')
plt.xlabel('x-axis')
plt.ylabel('y-axis')
plt.title("Input detections")
plt.savefig("results/input.jpg")

detections_2d = list(zip(ps1, ps2))

print(len(ps1),len(ps2),(detections_2d))

start = 0
stop = len(detections_2d)

dummy_color_histogram = [np.ones((128,))]*3
dummy_bbox = [0,0,1,1]

detections = []
for index in range(start, stop):
    for position in detections_2d[index]:
        detections.append(mot3d.Detection2D(index, position, 
                                            color_histogram=dummy_color_histogram,
                                            bbox=dummy_bbox))

print("Building the graph..")

weight_distance = lambda d1, d2: wf.weight_distance_detections_2d(d1, d2,
                                                                    sigma_jump=1, sigma_distance=10,
                                                                    sigma_color_histogram=0.3, sigma_box_size=0.3,
                                                                    max_distance=15,
                                                                    use_color_histogram=True, use_bbox=True)

weight_confidence = lambda d: wf.weight_confidence_detections_2d(d, mul=1, bias=0)
print(len(detections))
g = mot3d.build_graph(detections, weight_source_sink=0.1,
                        max_jump=4, verbose=True,
                        weight_confidence=weight_confidence,
                        weight_distance=weight_distance)
if g is None:
    raise RuntimeError("There is not a single path between sink and source nodes!") 
    
plt.figure()
mot3d.plot_graph(g)
plt.title("Min-cost max-flow graph")
plt.savefig("results/graph_detections.jpg")        

print("-"*30)
print("Solver first pass on detections..")
print("-"*30)
trajectories = mot3d.solve_graph(g, verbose=True, method='muSSP')  

plt.figure()
mot3d.plot_trajectories(trajectories)
plt.title("Extracted Detections")
plt.savefig("results/output_detections.jpg") 

# Breaking down trajectories into tracklets
tracklets = []
for traj in trajectories:
    tracklets += mot3d.split_trajectory_modulo(traj, length=6)

tracklets = mot3d.remove_short_trajectories(tracklets, th_length=2)

plt.figure()
mot3d.plot_trajectories(tracklets)
plt.title("Extracted Tracklets before Assoc")
plt.savefig("results/raw_tracklets.jpg") 

    
print("Building graph on tracklets..")
    
detections_tracklets = [mot3d.DetectionTracklet2D(tracklet) for tracklet in tracklets]

weight_distance_t = lambda t1, t2: wf.weight_distance_tracklets_2d(t1, t2, max_distance=None,
                                                                    sigma_color_histogram=0.3, sigma_motion=50, alpha=0.7,
                                                                    cutoff_motion=0.1, cutoff_appearance=0.1,
                                                                    use_color_histogram=True)

weight_confidence_t = lambda t: wf.weight_confidence_tracklets_2d(t, mul=1, bias=0)

g = mot3d.build_graph(detections_tracklets, weight_source_sink=0.1,
                        max_jump=20, verbose=True,
                        weight_confidence=weight_confidence_t,
                        weight_distance=weight_distance_t)    
if g is None:
    raise RuntimeError("There is not a single path between sink and source nodes!") 
    
plt.figure()
mot3d.plot_graph(g)
plt.title("Min-cost max-flow graph on tracklets")
plt.savefig("results/graph_tracklets.jpg")  

    
print("-"*30)
print("Solver second pass on tracklets..")
print("-"*30)
trajectories = mot3d.solve_graph(g, verbose=True, method='muSSP')   

plt.figure()
mot3d.plot_trajectories(trajectories)
plt.title("Output (muSSP)")
plt.savefig("results/output_mussp.jpg")
 

Loading detections..
13 13 [(array([17.26080771, 73.69815783]), array([29.92747438, 81.09026199])), (array([22.67206313, 69.58560372]), array([31.22617568, 76.76125766])), (array([28.29976875, 65.25659939]), array([32.95777741, 70.05130095])), (array([32.62877308, 61.36049549]), array([34.68937914, 63.34134424])), (array([36.19136283, 57.79548662]), array([36.68667866, 56.11169208])), (array([41.28678174, 54.866989  ]), array([39.01838347, 50.57078147])), (array([46.48158694, 51.62023575]), array([42.48158694, 43.86082476])), (array([52.32574278, 47.72413186]), array([44.6460891 , 37.58376848])), (array([56.87119732, 44.91027904]), array([47.2434917 , 30.44091134])), (array([60.76730122, 40.14837428]), array([50.05734451, 25.02965593])), (array([66.17855663, 35.1700193 ]), array([52.00539646, 19.18550008])), (array([71.58981204, 32.35616649]), array([53.52054797, 13.99069489])), (array([76.56816702, 29.97521411]), array([55.90150036,  8.36298926]))]
Building the graph..
26
Number of fr

100%|██████████| 54/54 [00:00<00:00, 5696.49it/s]

Number of post-pre nodes edges created: 69
Fraction of edges per detection: 5.653846153846154



No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


------------------------------
Solver first pass on detections..
------------------------------
[muSSP] 0 out of 52 nodes unused
Graph solved in 0.0007s using muSSP.
# of dummy edges : 0
The overall time is 4.5e-05 s

cost path #1 -14.6224652
cost path #2 -14.2653056
The number of paths: 2, total cost is -28.8877708, final path cost is: -14.2653056. #path_set 2
Building graph on tracklets..
Number of tracklets: 4


100%|██████████| 10/10 [00:00<00:00, 5238.95it/s]
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Number of post-pre nodes edges created: 4
Fraction of edges per detection: 4.0
------------------------------
Solver second pass on tracklets..
------------------------------
[muSSP] 0 out of 8 nodes unused
Graph solved in 0.0003s using muSSP.
# of dummy edges : 0
The overall time is 2.6e-05 s

cost path #1 -1.8905373
cost path #2 -1.8870663
The number of paths: 2, total cost is -3.7776036, final path cost is: -1.8870663. #path_set 2
